In [2]:
import pyspark
from pyspark.sql.functions import col,when,size,min, max,length
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.master('local[*]').appName('topic_modelling').getOrCreate()

In [4]:
df_parquet = spark.read.parquet("part-00000-a8a717e8-bf83-42e3-a0b7-084971053d99-c000.snappy.parquet")

In [5]:
# create a column and put in the detected language code
from langdetect import detect
getCode = lambda x:detect(x)
sent = "this is eng"
getCode(sent)

'en'

In [6]:
# create a column and put in the detected language code
from langdetect import detect
# app_name_result = df_parquet.select(col('description')).collect()
# # create udf function to add a new column
# import pyspark.sql.functions as F
# from pyspark.sql.types import *

# def detect_lang(value):
#   return detect(value)

# #convert to a UDF Function by passing in the function and return type of function
# udf_detect_lang = F.udf(detect_lang, StringType())
# df_with_code = df_parquet.withColumn('lang_code', udf_detect_lang(df_parquet.description))
# df_with_code.show(5)

# easier method to convert to pandas then apply then function
df_parquet_pd = df_parquet.toPandas()
df_parquet_pd['lang_code'] = df_parquet_pd['description'].apply(getCode)
df_parquet_pd

,app_name,bundle,description,genres,os_platform,lang_code
0,Black Box - Movie Listing,958957112,Black Box Movie Listing App is a collection li...,"[Entertainment, Magazines & Newspapers]",IOS,en
1,Knife Throwing Max,1491396285,you will have fun in this game\n100+ challengi...,"[Games_Sports, Games_Role Playing]",IOS,en
2,爆料公社,1387765782,全台最大社群「爆料公社」APP 上線啦，我們把社群解放了，不只有勁爆的爆料內容，還貼心的加上...,[Social Networking],IOS,zh-tw
3,Lidow - Photo Editor & Collage,894532288,layout/grid/collage、Square/no crop for instagr...,[Photo & Video],IOS,en
4,Vidstitch Frames for Instagram,712908978,"■ #Vidstitch\n■ Featured on 148apps, Stelapps,...","[Photo & Video, Social Networking]",IOS,en
...,...,...,...,...,...,...
860,Barber Hair Salon & Beard Makeover,com.hmg.haircutgames.beardsalon.barberhaircut,Barber hair makeover salon game is for those w...,"[ENTERTAINMENT, FAMILY_PRETEND]",ANDROID,en
861,DJ Remix Offline 2020,com.mpro.djremixoffline2020,Dj Remix Offline 2020 Mp3 merupakan aplikasi m...,[MUSIC_AND_AUDIO],ANDROID,id
862,Sahih Bukhari (English),com.bangladroid.sahihbukhari,We have made this application from the book of...,[BOOKS_AND_REFERENCE],ANDROID,en
863,Two Minute English,com.astrobix.twominuteenglish,Improve your spoken English rapidly with the h...,[EDUCATION],ANDROID,en


In [7]:
# load with only English data
df_parquet_en = df_parquet_pd.loc[df_parquet_pd['lang_code']=='en']
df_parquet_en.shape

(687, 6)

In [8]:
row1 = df_parquet_en['description'][0]

In [17]:
from rake_nltk import Metric, Rake

# Uses stopwords for english from NLTK, and all puntuation characters.
# r = Rake()
r = Rake(min_length=1, max_length=3)

r.extract_keywords_from_text(row1)

r.get_ranked_phrases() # To get keyword phrases ranked highest to lowest.

['https :// reign',
 'http :// reign',
 'movie trailer online',
 'black box app',
 'newly released movies',
 'app pulls movie',
 'tv show information',
 'social media platforms',
 'gain premium access',
 '3 different subscriptions',
 'renewal within 24',
 'informs app users',
 'wish list movies',
 'events privacy policy',
 'movie collection',
 'tv shows',
 'least 24',
 'diverse events',
 'detailed information',
 'app subscriptions',
 'view details',
 'video list',
 'specific areas',
 'simple way',
 'release date',
 'one month',
 'offline list',
 'local plays',
 'imdb website',
 'general population',
 'every month',
 'diverse communities',
 'current period',
 'collection list',
 'available posters',
 '99 usd',
 '), actors',
 'news postings',
 'news content',
 'international news',
 'media section',
 'itunes account',
 'watch later',
 'view director',
 'subscription anytime',
 'purchase one',
 'occurring around',
 'hours prior',
 'account section',
 'php terms',
 'app',
 'movies',
 'user

In [15]:
from rake_nltk import Metric, Rake

# Uses stopwords for english from NLTK, and all puntuation characters.
# r = Rake()
r = Rake(min_length=2, max_length=4,ranking_metric=Metric.WORD_FREQUENCY)

r.extract_keywords_from_text(row1)

r.get_ranked_phrases() # To get keyword phrases ranked highest to lowest.

['monthly ad removal subscription',
 'annual ad removal subscription',
 'app pulls movie',
 'wish list movies',
 'subscription also removes advertisement',
 'informs app users',
 'collection list',
 'black box app',
 'app subscriptions',
 'video list',
 'tv show information',
 'subscription anytime',
 'offline list',
 'movie trailer online',
 'movie collection',
 'https :// reign',
 'http :// reign',
 'user purchases february 22nd',
 'social media platforms',
 'renewal within 24',
 'one month',
 'news postings',
 'news content',
 'newly released movies',
 'media section',
 'international news',
 'events privacy policy',
 'diverse events',
 'automatically renews unless auto',
 'account section',
 '3 different subscriptions',
 'view director',
 'view details',
 'tv shows',
 'purchase one',
 'least 24',
 'itunes account',
 'gain premium access',
 'every month',
 'diverse communities',
 'detailed information',
 'watch later',
 'specific areas',
 'simple way',
 'release date',
 'php terms',

In [16]:
from rake_nltk import Metric, Rake

# Uses stopwords for english from NLTK, and all puntuation characters.
# r = Rake()
r = Rake(min_length=2, max_length=4,ranking_metric=Metric.WORD_DEGREE)

r.extract_keywords_from_text(row1)

r.get_ranked_phrases() # To get keyword phrases ranked highest to lowest.

['monthly ad removal subscription',
 'annual ad removal subscription',
 'subscription also removes advertisement',
 'app pulls movie',
 'wish list movies',
 'informs app users',
 'black box app',
 'user purchases february 22nd',
 'subscription anytime',
 'automatically renews unless auto',
 'app subscriptions',
 'https :// reign',
 'http :// reign',
 'movie trailer online',
 'tv show information',
 'collection list',
 'newly released movies',
 'movie collection',
 'video list',
 'social media platforms',
 'renewal within 24',
 'offline list',
 'events privacy policy',
 '3 different subscriptions',
 'media section',
 'gain premium access',
 'diverse events',
 'one month',
 'news postings',
 'news content',
 'international news',
 'account section',
 'tv shows',
 'least 24',
 'detailed information',
 'view director',
 'view details',
 'purchase one',
 'itunes account',
 'every month',
 'diverse communities',
 'watch later',
 'specific areas',
 'simple way',
 'release date',
 'php terms',